#Lab 03 - SQL Query Pada DataFrame.ipynb

SQL adalah salah satu bahasa populer untuk pemrosesan dan analisis data. Spark mendukung SQL untuk memproses DataFrame. Dalam latihan ini kita akan menggunakan spark SQL tanpa database.


In [1]:
%pip install pyspark

Import packages

In [2]:
from pyspark.sql import SparkSession

Inisialisasi spark session untuk berinteraksi dengan Spark cluster.

In [3]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

Download dataset

In [4]:
!wget https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz

--2025-08-11 04:37:10--  https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz [following]
--2025-08-11 04:37:11--  https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3175443 (3.0M) [application/octet-stream]
Saving to: ‘application_record_header.csv.gz’

application_record_ 100%[===================>]   3.03M  --.-KB/s    in 0.06s

Load ke dataframe

In [5]:
df = spark.read.csv("application_record_header.csv.gz", header=True, inferSchema=True)

Tampilkan 5 baris pertama

In [6]:
df.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

## Membuat temporary table/view

Sebelum menggunakan SQL, kita perlu membuat temporary table dari dataframe yang akan kita olah.

Gunakan fungsi `createOrReplaceTempView(nama_tabel)` pada dataframe tersebut.

In [7]:
df.createOrReplaceTempView("app_record")

## Menjalankan perintah SQL

Selanjutnya untuk mengakses DataFrame dengan SQL, kita bisa menggunakan nama tabel yang sudah kita definisikan dalam SQL statement.

Untuk mengeksekusi SQL statement, kita gunakan fungsi `sql(sqlstatement)` pada object spark session.

perintah `sql()` mengembalikan sebuah DataFrame. Untuk menampilkan ke layar, kita perlu gunakan fungsi `show()` seperti DataFrame pada umumnya.

In [8]:
spark.sql("select avg(DAYS_BIRTH) from app_record").show()

+-------------------+
|    avg(DAYS_BIRTH)|
+-------------------+
|-15997.904648654565|
+-------------------+



In [9]:
spark.sql("select * from app_record limit 5").show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

## Join dan Agregasi DataFrame dengan SQL

Kita akan melakukan join salah satu kolom dengan data referensi dan melakukan agregasi. Sebelumnya kita buat dataframe referensi dan membuat temporary viewnya

In [10]:
spark.sql("select distinct NAME_EDUCATION_TYPE from app_record").show(truncate=False)

+-----------------------------+
|NAME_EDUCATION_TYPE          |
+-----------------------------+
|Academic degree              |
|Incomplete higher            |
|Secondary / secondary special|
|Lower secondary              |
|Higher education             |
+-----------------------------+



In [11]:
mydata = (
    ('Lower secondary',1),
    ('Secondary / secondary special',2),
    ('Academic degree',3),
    ('Incomplete higher',4),
    ('Higher education',5))

ref_edu = spark.createDataFrame(mydata).toDF("NAME_EDUCATION_TYPE", "EDU_LEVEL")
ref_edu.createOrReplaceTempView("ref_edu")
spark.sql("select * from ref_edu").show()

+--------------------+---------+
| NAME_EDUCATION_TYPE|EDU_LEVEL|
+--------------------+---------+
|     Lower secondary|        1|
|Secondary / secon...|        2|
|     Academic degree|        3|
|   Incomplete higher|        4|
|    Higher education|        5|
+--------------------+---------+



Kita lakukan join, agregat, kemudian kita tampilkan hasilnya ke layar

In [15]:
agg_edu = spark.sql("""SELECT edu_level, count(1) as number_of_app FROM
              (SELECT ref_edu.EDU_LEVEL as edu_level
                FROM app_record LEFT JOIN ref_edu
                ON app_record.NAME_EDUCATION_TYPE=ref_edu.NAME_EDUCATION_TYPE)
             GROUP BY edu_level SORT BY edu_level""")

agg_edu.show()

+---------+-------------+
|edu_level|number_of_app|
+---------+-------------+
|        1|         4051|
|        2|       301821|
|        3|          312|
|        4|        14851|
|        5|       117522|
+---------+-------------+



## Menyimpan ke database

Untuk mengupload konten DataFrame ke database, gunakan perintah `DataFrame.write.saveAsTable(name=nama_tabel_db)`

Kita juga dapat menentukan mode penulisannya : `append`, `overwrite`, `error`, `errorifexists`, `ignore`.

In [16]:
agg_edu.write.saveAsTable(name="aggregated_edu", mode="overwrite")

## Menampilkan metadata tabel

Kita bisa melihat atribut tabel yang dihasilkan dengan menampilkan metadatanya, menggunakan perintah `describe formatted`

In [13]:
spark.sql("describe formatted aggregated_edu").show(truncate = False)

+----------------------------+--------------------------------------------+-------+
|col_name                    |data_type                                   |comment|
+----------------------------+--------------------------------------------+-------+
|edu_level                   |bigint                                      |NULL   |
|number_of_app               |bigint                                      |NULL   |
|                            |                                            |       |
|# Detailed Table Information|                                            |       |
|Catalog                     |spark_catalog                               |       |
|Database                    |default                                     |       |
|Table                       |aggregated_edu                              |       |
|Created Time                |Mon Aug 11 04:37:42 UTC 2025                |       |
|Last Access                 |UNKNOWN                                     | 

Karena kita tidak menggunakan koneksi ke database, perintah `DataFrame.write.saveAsTable()` akan menyimpannya sebagai file parquet ke sebuah direktori di distributed storage yang digunakan.

Lokasi direktorinya dapat dilihat dari atribut `Location`, dalam hal ini `file:/content/spark-warehouse/aggregated_edu`

In [14]:
%ls -l /content/spark-warehouse/aggregated_edu/

total 4
-rw-r--r-- 1 root root 824 Aug 11 04:37 part-00000-c1a568c8-696c-496d-a009-9201c52eb196-c000.snappy.parquet
-rw-r--r-- 1 root root   0 Aug 11 04:37 _SUCCESS
